In [1]:
import os
root_dir = ""
root_dir = os.getcwd()

In [2]:
code_dir = root_dir + "/" + "Fortran_Code/Section_Cellular_Automata"

In [3]:
os.chdir(code_dir)

In [4]:
#build_status = os.system("fpm build 2>/dev/null")
build_status = os.system("fpm build")

Project is up to date


In [5]:
exec_status = \
    os.system("fpm run > output.txt 2> /dev/null")

In [6]:
!pwd

/home/haxor/Documents/Engineering/MKDynamics_website_V2/Sections/Computer_Programming/Fortran/Fortran_MOOC/Fortran_Code/Section_Cellular_Automata


In [7]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import subprocess

In [8]:

def run_fortran_ca(rule=90, nr_cells=50, max_steps=100):

    # Call the Fortran program using FPM
    # The correct way to call FPM is to use:
    # fpm run -- rule_nr nr_cells max_steps
    # from subprocess.run
    # This will execute the Fortran program with the specified parameters
    if not isinstance(rule, int) or rule < 0:
        raise ValueError("Rule must be a non-negative integer.")
    if not isinstance(nr_cells, int) or nr_cells <= 0:
        raise ValueError("Number of cells must be a positive integer.")
    if not isinstance(max_steps, int) or max_steps <= 0:
        raise ValueError("Maximum steps must be a positive integer.")

    # Run the Fortran program with the specified parameters
    # The Fortran program expects the rule number, number of cells, and max steps as command line arguments
    # redirect the output to 'output.txt' and errors to /dev/null
    process = subprocess.run(
        f"fpm run -- {str(rule)} {str(nr_cells)} {str(max_steps)} > output.txt 2> /dev/null", shell=True, text=True
    )

    # Check if the process was successful
    if process.returncode != 0:
        raise RuntimeError("Fortran program failed to run. Please check the Fortran code and the parameters provided.")

    # Check if the output file was created
    if not os.path.exists("output.txt"):
        raise FileNotFoundError("Output file 'output.txt' was not created. Please check if the Fortran program ran successfully.")

    # Read the output file 'output.txt' generated by the Fortran program
    with open("output.txt", "r") as f:
        output = f.read().splitlines()

    # Check if the output is empty
    if not output:
        raise ValueError("The output file is empty. Please check if the Fortran program ran successfully.")
    
    # Parse the output: skip rule print, collect lines that are the grid states
    history = []
    for i, line in enumerate(output):
        # Only remove line endings, preserve all spaces
        cleaned_line = line.rstrip('\r\n')
        # Use {' ', 'X'} if your Fortran uses 'X'; otherwise {' ', '*'}
        is_valid = len(cleaned_line) == nr_cells and set(cleaned_line).issubset({' ', 'X'})
        # Debug logging
        #print(f"Line {i+1}: '{cleaned_line}' (Length: {len(cleaned_line)}, Valid: {is_valid})")
        if is_valid:
            # Convert ' ' to 0, 'X' to 1 (or '*' to 1 if using '*')
            row = np.array([1 if c == 'X' else 0 for c in cleaned_line])
            #print(f"Parsed row: {row}")
            # Append the row to the history
            history.append(row)
    
    # Stack into 2D array (steps + 1 rows, nr_cells columns)
    history = np.array(history)
    # Check expected vs actual
    expected_steps = max_steps + 1
    if history.shape[0] != expected_steps:
        print(f"Warning: Expected {expected_steps} steps, parsed {history.shape[0]}")
    return history

In [9]:
run_fortran_ca(rule=90, nr_cells=50, max_steps=100)

array([[1, 1, 0, ..., 0, 1, 0],
       [1, 1, 0, ..., 1, 0, 0],
       [1, 1, 0, ..., 0, 1, 1],
       ...,
       [0, 1, 0, ..., 1, 0, 0],
       [1, 0, 0, ..., 1, 1, 0],
       [0, 1, 1, ..., 0, 1, 0]], shape=(101, 50))

In [10]:
# Main animation function
def animate_ca(rule=90, width=50, steps=100):
    history = run_fortran_ca(rule, width, steps)
    
    fig, ax = plt.subplots(figsize=(7, 7))
    ax.set_axis_off()
    img = ax.imshow(history, cmap='binary', interpolation='nearest', aspect='auto')
    
    def animate(frame):
        img.set_data(history[:frame + 1])
        ax.set_title(f'Elementary Cellular Automaton - Rule {rule}\nStep: {frame}')
        return [img]
    
    anim = FuncAnimation(fig, animate, frames=history.shape[0], interval=100, blit=True)
    plt.close(fig)
    return HTML(anim.to_jshtml())


In [11]:

animate_ca(rule=90, width=50, steps=100)